In [1]:
from pathlib import Path
import pandas as pd

PROJECT = Path.home() / "emt_network"
RAW = PROJECT / "data" / "raw"
PROCESSED = PROJECT / "data" / "processed"
RAW.mkdir(parents=True, exist_ok=True)
PROCESSED.mkdir(parents=True, exist_ok=True)

print("Project:", PROJECT)
print("Raw:", RAW)
print("Processed:", PROCESSED)


Project: /home/sameeksha/emt_network
Raw: /home/sameeksha/emt_network/data/raw
Processed: /home/sameeksha/emt_network/data/processed


In [2]:
import requests

def download(url: str, out_path: Path, chunk_size: int = 1024 * 1024) -> Path:
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()
        with open(out_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
    return out_path


In [5]:
import sys
!{sys.executable} -m pip install -U xenaPython


In [6]:
import xenaPython as xena
hub = "https://tcga.xenahubs.net"


/home/sameeksha/emt_network/.venv/lib/python3.12/site-packages/xenaPython/__init__.py:110: FutureWarning: Possible nested set at position 7
  re.sub(r"^[^[]+[[]([^]]*)[]].*$", r"\1", query, flags=re.DOTALL))


In [7]:
dataset = "TCGA.BRCA.sampleMap/HiSeqV2"


In [9]:
samples = xena.dataset_samples(hub, dataset, 10000)
len(samples), samples[:10]


(1218,
 ['TCGA-AR-A5QQ-01',
  'TCGA-D8-A1JA-01',
  'TCGA-BH-A0BQ-01',
  'TCGA-BH-A0BT-01',
  'TCGA-A8-A06X-01',
  'TCGA-A8-A096-01',
  'TCGA-BH-A0C7-01',
  'TCGA-AC-A5XU-01',
  'TCGA-PE-A5DE-01',
  'TCGA-PE-A5DC-01'])

In [10]:
emt_genes = ["CDH1","VIM","EPCAM","ZEB1","ZEB2","SNAI1","SNAI2","TWIST1"]
expr = xena.dataset_gene_probes_values(hub, dataset, samples[:50], emt_genes)
type(expr), len(expr)


(list, 2)

In [11]:
probes = expr[0]
values = expr[1]

type(probes), type(values), len(probes), len(values), len(values[0])

(dict, list, 2, 8, 50)

In [19]:

import pandas as pd

gene_names = expr[0]["name"]
values = expr[1]

df_small = pd.DataFrame(values, index=gene_names, columns=samples[:50])
df_small.shape, df_small.head()




((8, 50),
        TCGA-AR-A5QQ-01  TCGA-D8-A1JA-01  TCGA-BH-A0BQ-01  TCGA-BH-A0BT-01  \
 VIM             15.420           13.260           14.810           13.770   
 ZEB1             7.948            9.733            9.988            8.679   
 CDH1            11.960           14.850           12.530           12.650   
 EPCAM            8.979           12.870           11.860           11.350   
 ZEB2             9.055           10.060            9.181            7.925   
 
        TCGA-A8-A06X-01  TCGA-A8-A096-01  TCGA-BH-A0C7-01  TCGA-AC-A5XU-01  \
 VIM             13.320            14.77           13.590           14.440   
 ZEB1             8.732            10.12           10.010            8.953   
 CDH1            13.810            12.25           14.360           13.780   
 EPCAM           14.010            11.00           12.670           12.820   
 ZEB2             8.682            10.32            9.434            8.768   
 
        TCGA-PE-A5DE-01  TCGA-PE-A5DC-01  ...  TCG